In [6]:
import pandas as pd
import numpy as np
!pip install openpyxl
# Load data from Excel
file_path = 'NIFTY 50-01-01-2023-to-31-12-2023.csv'
df = pd.read_csv(file_path)
print("Dataset Loaded Successfully:")


df.head()

Dataset Loaded Successfully:


,Date,Open,High,Low,Close,Shares Traded,Turnover (₹ Cr)
0,02-JAN-2023,18131.70,18215.15,18086.50,18197.45,256073979,11793.06
1,03-JAN-2023,18163.20,18251.95,18149.80,18232.55,208699772,13680.91
2,04-JAN-2023,18230.65,18243.00,18020.60,18042.95,235162202,16687.71
3,05-JAN-2023,18101.95,18120.30,17892.60,17992.15,269949738,22108.53
4,06-JAN-2023,18008.05,18047.40,17795.55,17859.45,238220578,17495.25


In [7]:
# Rename the columns to match the standard format
new_column_names = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']
df.columns = new_column_names

# Display renamed columns
print("Updated Columns:")
print(df.columns)

Updated Columns:
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover'], dtype='object')


In [8]:
# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Drop rows with missing values
df.dropna(inplace=True)

# Sort the data by Date in ascending order
df.sort_values(by='Date', inplace=True)

# Display the preprocessed data
print("Preprocessed Data:")
print(df.head())

Preprocessed Data:
        Date      Open      High       Low     Close     Volume  Turnover
0 2023-01-02  18131.70  18215.15  18086.50  18197.45  256073979  11793.06
1 2023-01-03  18163.20  18251.95  18149.80  18232.55  208699772  13680.91
2 2023-01-04  18230.65  18243.00  18020.60  18042.95  235162202  16687.71
3 2023-01-05  18101.95  18120.30  17892.60  17992.15  269949738  22108.53
4 2023-01-06  18008.05  18047.40  17795.55  17859.45  238220578  17495.25


C:\Users\Rishith\AppData\Local\Temp\ipykernel_16644\2402149932.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [9]:
# Calculate Daily Returns
df['Daily_Return'] = df['Close'].pct_change()

# Calculate Moving Averages (10-day and 50-day)
df['MA_10'] = df['Close'].rolling(window=10).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()

# Calculate Volatility (10-day rolling standard deviation of returns)
df['Volatility'] = df['Daily_Return'].rolling(window=10).std()

# Display the DataFrame with new metrics
print("Data with Metrics:")
print(df[['Date', 'Close', 'Daily_Return', 'MA_10', 'MA_50', 'Volatility']].head(15))


Data with Metrics:
         Date     Close  Daily_Return      MA_10  MA_50  Volatility
0  2023-01-02  18197.45           NaN        NaN    NaN         NaN
1  2023-01-03  18232.55      0.001929        NaN    NaN         NaN
2  2023-01-04  18042.95     -0.010399        NaN    NaN         NaN
3  2023-01-05  17992.15     -0.002816        NaN    NaN         NaN
4  2023-01-06  17859.45     -0.007375        NaN    NaN         NaN
5  2023-01-09  18101.20      0.013536        NaN    NaN         NaN
6  2023-01-10  17914.15     -0.010334        NaN    NaN         NaN
7  2023-01-11  17895.70     -0.001030        NaN    NaN         NaN
8  2023-01-12  17858.20     -0.002095        NaN    NaN         NaN
9  2023-01-13  17956.60      0.005510  18005.040    NaN         NaN
10 2023-01-16  17894.85     -0.003439  17974.780    NaN    0.007338
11 2023-01-17  18053.30      0.008855  17956.855    NaN    0.008009
12 2023-01-18  18165.35      0.006207  17969.095    NaN    0.007542
13 2023-01-19  18107.85     -

In [11]:
# Define thresholds for detecting anomalies
return_threshold = 0.02  # Adjusted: 2% daily return
volatility_threshold = 0.01  # Adjusted: 1% volatility

# Identify anomalies
df['Anomaly'] = np.where(
    (abs(df['Daily_Return']) > return_threshold) | 
    (df['Volatility'] > volatility_threshold),
    True,  # Mark as anomaly
    False  # Normal data
)

# Display anomalies
anomalies = df[df['Anomaly'] == True]
print("Anomalies Detected:")
print(anomalies[['Date', 'Close', 'Daily_Return', 'Volatility']])

# Debugging if no anomalies are found
if anomalies.empty:
    print("No anomalies detected. Try relaxing the thresholds.")
    print("Sample data for debugging:")
    print(df[['Date', 'Close', 'Daily_Return', 'Volatility']].tail(10))



Anomalies Detected:
          Date    Close  Daily_Return  Volatility
227 2023-12-04  20686.8      0.020668    0.006724


In [12]:
# Export the preprocessed data and anomalies to separate Excel sheets
output_file = 'nifty_stock_analysis_output.xlsx'
with pd.ExcelWriter(output_file) as writer:
    df.to_excel(writer, sheet_name='Processed Data', index=False)
    anomalies.to_excel(writer, sheet_name='Anomalies', index=False)

print(f"Results exported successfully to {output_file}")


Results exported successfully to nifty_stock_analysis_output.xlsx
